# Zika VLP packing — Packmol vs Warp-pack

Reproducible notebook to build the Zika VLP protein–membrane system and compare **Packmol** vs **Warp-pack** outputs using the same `.inp` files.

Notes:
- Packmol is the reference Fortran tool.
- Warp-pack (this repo) reads the same `.inp` and runs in Rust; export uses Rust writers.


## Install tools

This cell installs **warp-md** and Packmol (if conda is available).


In [ ]:
import shutil, subprocess, sys

subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "pip"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "warp-md"])

# Optional companion package (if available)
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "warp-mdm"])
except subprocess.CalledProcessError as exc:
    print("warp-mdm install failed; verify package name/version.")

# QC dependencies
subprocess.check_call([sys.executable, "-m", "pip", "install", "MDAnalysis"])

# Visualization dependencies
subprocess.check_call([sys.executable, "-m", "pip", "install", "py3Dmol"])

if shutil.which("conda"):
    subprocess.check_call(["conda", "install", "-y", "-c", "conda-forge", "packmol"])
else:
    print("conda not found; install Packmol manually if you want Packmol runs.")

## Project layout

We create a working directory tree under `zika_pack/`.


In [ ]:
from pathlib import Path

root = Path("zika_pack")
(root / "00_raw").mkdir(parents=True, exist_ok=True)
(root / "01_protein").mkdir(parents=True, exist_ok=True)
(root / "02_lipids").mkdir(parents=True, exist_ok=True)
(root / "03_packmol").mkdir(parents=True, exist_ok=True)
(root / "04_qc").mkdir(parents=True, exist_ok=True)
print("Created", root)


## Step 1 — Fetch PDB 6CO8


In [ ]:
import urllib.request
from pathlib import Path

url = "https://files.rcsb.org/download/6CO8.pdb"
out = Path("zika_pack/00_raw/6co8.pdb")
if not out.exists():
    urllib.request.urlretrieve(url, out)
print("Downloaded", out)


## Step 2 — Apply BIOMT (expand envelope)


In [ ]:
from pathlib import Path

script = r'''#!/usr/bin/env python3
# Expand a PDB file using BIOMT matrices from REMARK 350.

import re
import sys
from math import isfinite

biomt_re = re.compile(
    r"^REMARK\s+350\s+BIOMT(\d)\s+(\d+)\s+([-0-9.Ee]+)\s+([-0-9.Ee]+)\s+([-0-9.Ee]+)\s+([-0-9.Ee]+)\s*$"
)


def parse_biomt(lines):
    ops = {}
    for line in lines:
        m = biomt_re.match(line)
        if not m:
            continue
        row = int(m.group(1)) - 1
        idx = int(m.group(2))
        vals = list(map(float, m.groups()[2:]))
        if idx not in ops:
            ops[idx] = [[0.0, 0.0, 0.0, 0.0] for _ in range(3)]
        ops[idx][row] = vals

    complete = []
    for idx in sorted(ops):
        mat = ops[idx]
        if all(all(isfinite(x) for x in row) for row in mat):
            complete.append(mat)

    if not complete:
        raise RuntimeError("No BIOMT matrices found. Is this a PDB with REMARK 350 BIOMT lines?")

    return complete


def transform_xyz(mat, x, y, z):
    xp = mat[0][0] * x + mat[0][1] * y + mat[0][2] * z + mat[0][3]
    yp = mat[1][0] * x + mat[1][1] * y + mat[1][2] * z + mat[1][3]
    zp = mat[2][0] * x + mat[2][1] * y + mat[2][2] * z + mat[2][3]
    return xp, yp, zp


def main(inp, outp):
    with open(inp, "r", encoding="utf-8", errors="replace") as f:
        lines = f.readlines()

    biomts = parse_biomt(lines)
    atom_lines = [ln for ln in lines if ln.startswith(("ATOM  ", "HETATM"))]
    header = [ln for ln in lines if not ln.startswith(("ATOM  ", "HETATM"))]

    out = []
    out.extend(header)

    serial = 1
    for op_i, mat in enumerate(biomts, start=1):
        chain_tag = chr(((op_i - 1) % 26) + ord("A"))
        for ln in atom_lines:
            x = float(ln[30:38])
            y = float(ln[38:46])
            z = float(ln[46:54])
            xp, yp, zp = transform_xyz(mat, x, y, z)

            ln2 = list(ln)
            ln2[6:11] = f"{serial:5d}"
            ln2[21] = chain_tag
            ln2[30:38] = f"{xp:8.3f}"
            ln2[38:46] = f"{yp:8.3f}"
            ln2[46:54] = f"{zp:8.3f}"
            out.append("".join(ln2))
            serial += 1

    out.append("END
")

    with open(outp, "w", encoding="utf-8") as f:
        f.writelines(out)


if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: python apply_biomt.py input.pdb output.pdb", file=sys.stderr)
        sys.exit(2)
    main(sys.argv[1], sys.argv[2])
'''
path = Path("zika_pack/01_protein/apply_biomt.py")
path.write_text(script)
print("Wrote", path)


In [ ]:
import subprocess

subprocess.check_call(["python", "zika_pack/01_protein/apply_biomt.py", "zika_pack/00_raw/6co8.pdb", "zika_pack/01_protein/zika_env_full.pdb"])


## Step 3 — Center the envelope


In [ ]:
from pathlib import Path

script = r'''#!/usr/bin/env python3
# Center ATOM/HETATM coordinates of a PDB at the origin.

import sys


def main(inp, outp):
    atoms = []
    lines = []
    with open(inp) as f:
        for ln in f:
            lines.append(ln)
            if ln.startswith(("ATOM  ", "HETATM")):
                x = float(ln[30:38])
                y = float(ln[38:46])
                z = float(ln[46:54])
                atoms.append((x, y, z))

    cx = sum(a[0] for a in atoms) / len(atoms)
    cy = sum(a[1] for a in atoms) / len(atoms)
    cz = sum(a[2] for a in atoms) / len(atoms)

    out = []
    for ln in lines:
        if ln.startswith(("ATOM  ", "HETATM")):
            x = float(ln[30:38]) - cx
            y = float(ln[38:46]) - cy
            z = float(ln[46:54]) - cz
            ln2 = list(ln)
            ln2[30:38] = f"{x:8.3f}"
            ln2[38:46] = f"{y:8.3f}"
            ln2[46:54] = f"{z:8.3f}"
            out.append("".join(ln2))
        else:
            out.append(ln)

    with open(outp, "w") as f:
        f.writelines(out)


if __name__ == "__main__":
    main(sys.argv[1], sys.argv[2])
'''
path = Path("zika_pack/01_protein/center_pdb.py")
path.write_text(script)
print("Wrote", path)


In [ ]:
import subprocess

subprocess.check_call(["python", "zika_pack/01_protein/center_pdb.py", "zika_pack/01_protein/zika_env_full.pdb", "zika_pack/01_protein/zika_env_full_centered.pdb"])


## Step 4 — Lipid templates

Put one template PDB for each lipid in `zika_pack/02_lipids/`:

- `POPC.pdb`
- `POPE.pdb`
- `POPS.pdb`

Identify the phosphate atom index and a tail-tip atom index in each template.


### Download lipid PDBs (SIRAH bundle)

The SIRAH bundles include lipid template PDBs under `sirah_*/PDB/`.

Download from the official release pages:
- GROMACS: https://github.com/SIRAHFF/documentation/releases/tag/GROMACS
- AMBER: https://github.com/SIRAHFF/documentation/releases/tag/AMBER

Example bundle names from the docs:
- `sirah_x2.2_20-07.ff.tar.gz` (GROMACS)
- `sirah_x2.3_24-07.amber.tar.gz` (AMBER)

After download, extract and copy the lipid PDBs:

```bash
tar -xzvf sirah_x2.2_20-07.ff.tar.gz
cp sirah_x2.2_20-07.ff/PDB/POPC.pdb zika_pack/02_lipids/
cp sirah_x2.2_20-07.ff/PDB/POPE.pdb zika_pack/02_lipids/
cp sirah_x2.2_20-07.ff/PDB/POPS.pdb zika_pack/02_lipids/
```


In [ ]:
import json
import urllib.request

def download_release_asset(tag, filename, out_path):
    api = f"https://api.github.com/repos/SIRAHFF/documentation/releases/tags/{tag}"
    with urllib.request.urlopen(api) as resp:
        data = json.load(resp)
    url = None
    for asset in data.get("assets", []):
        if asset.get("name") == filename:
            url = asset.get("browser_download_url")
            break
    if not url:
        raise RuntimeError(f"Asset {filename} not found in tag {tag}")
    print("Downloading:", url)
    urllib.request.urlretrieve(url, out_path)

download_release_asset("GROMACS", "sirah_x2.2_20-07.ff.tar.gz", "sirah_x2.2_20-07.ff.tar.gz")
download_release_asset("AMBER", "sirah_x2.3_24-07.amber.tar.gz", "sirah_x2.3_24-07.amber.tar.gz")


In [ ]:
from pathlib import Path

script = r'''#!/usr/bin/env python3
"""Extract a single residue (lipid) from a PDB and write it as a template.

Usage:
  python scripts/extract_lipid_pdb.py input.pdb output.pdb --resname POPC
  python scripts/extract_lipid_pdb.py input.pdb output.pdb --resname POPC --resid 12
  python scripts/extract_lipid_pdb.py input.pdb output.pdb --resname POPC --chain A
  python scripts/extract_lipid_pdb.py input.pdb output.pdb --list

Notes:
  - Reads ATOM/HETATM records only.
  - If --resid is omitted, the first matching residue is used.
  - Output is renumbered sequentially and ends with END.
"""

from __future__ import annotations

import argparse
from collections import Counter
from dataclasses import dataclass
from pathlib import Path
from typing import Iterable, List, Optional, Tuple


@dataclass(frozen=True)
class ResidueKey:
    resname: str
    chain: str
    resid: int


def iter_atoms(lines: Iterable[str]) -> Iterable[Tuple[int, ResidueKey, str]]:
    for line in lines:
        if not line.startswith(("ATOM  ", "HETATM")):
            continue
        try:
            resname = line[17:20].strip()
            chain = line[21].strip() or "A"
            resid = int(line[22:26].strip())
        except Exception:
            continue
        key = ResidueKey(resname=resname, chain=chain, resid=resid)
        yield resid, key, line.rstrip("\n")


def list_residues(lines: Iterable[str]) -> List[ResidueKey]:
    counts: Counter[ResidueKey] = Counter()
    for _, key, _ in iter_atoms(lines):
        counts[key] += 1
    for key, count in sorted(counts.items(), key=lambda kv: (kv[0].resname, kv[0].resid)):
        print(f"{key.resname:>4} chain={key.chain} resid={key.resid} atoms={count}")
    return list(counts.keys())


def select_residue(
    keys: List[ResidueKey],
    resname: Optional[str],
    resid: Optional[int],
    chain: Optional[str],
) -> ResidueKey:
    candidates = keys
    if resname:
        candidates = [k for k in candidates if k.resname == resname]
    if chain:
        candidates = [k for k in candidates if k.chain == chain]
    if resid is not None:
        candidates = [k for k in candidates if k.resid == resid]
    if not candidates:
        raise ValueError("No matching residue found. Use --list to inspect residues.")
    return candidates[0]


def renumber_atom(line: str, new_serial: int) -> str:
    # PDB columns 7-11 (1-based) are serial.
    serial = f"{new_serial:5d}"
    return f"{line[:6]}{serial}{line[11:]}"


def main() -> None:
    parser = argparse.ArgumentParser(description=__doc__)
    parser.add_argument("input", type=Path)
    parser.add_argument("output", type=Path)
    parser.add_argument("--resname", type=str, default=None, help="Residue name (e.g., POPC)")
    parser.add_argument("--resid", type=int, default=None, help="Residue ID number")
    parser.add_argument("--chain", type=str, default=None, help="Chain ID (single character)")
    parser.add_argument("--list", action="store_true", help="List residues and exit")
    args = parser.parse_args()

    lines = args.input.read_text().splitlines()

    if args.list:
        list_residues(lines)
        return

    if args.resname is None and args.resid is None:
        raise SystemExit("Provide --resname and/or --resid, or use --list.")

    keys = list_residues(lines)
    target = select_residue(keys, args.resname, args.resid, args.chain)

    out_lines: List[str] = []
    serial = 1
    for _, key, line in iter_atoms(lines):
        if key != target:
            continue
        out_lines.append(renumber_atom(line, serial))
        serial += 1
    if not out_lines:
        raise SystemExit("No atoms found for selected residue.")
    out_lines.append("END")

    args.output.write_text("\n".join(out_lines) + "\n")
    print(f"Wrote {args.output} with {serial - 1} atoms.")


if __name__ == "__main__":
    main()
'''
Path("scripts").mkdir(parents=True, exist_ok=True)
Path("scripts/extract_lipid_pdb.py").write_text(script)
print("Wrote scripts/extract_lipid_pdb.py")


### If you downloaded a bilayer (multi-lipid) PDB

Extract a single lipid as a template using the helper script:

```bash
python scripts/extract_lipid_pdb.py bilayer.pdb zika_pack/02_lipids/POPC.pdb --resname POPC
python scripts/extract_lipid_pdb.py bilayer.pdb zika_pack/02_lipids/POPE.pdb --resname POPE
python scripts/extract_lipid_pdb.py bilayer.pdb zika_pack/02_lipids/POPS.pdb --resname POPS
```

List residues first to confirm IDs:

```bash
python scripts/extract_lipid_pdb.py bilayer.pdb out.pdb --list
```


## Step 5 — Write Packmol inputs (cycle 1 + cycle 2)

Set the atom indices below, then generate the `.inp` files.


In [ ]:
PHOS_ATOM_INDEX = 1  # TODO: set phosphate atom index
TAIL_ATOM_INDEX = 2  # TODO: set tail-tip atom index


In [ ]:
from pathlib import Path

cycle1 = ftolerance 3.0
filetype pdb
output zika_pack_cycle1.pdb

packall
movebadrandom

maxit 40
nloop 100
movefrac 0.01

restart_to zika_pack_cycle1.restart

# --- fixed protein envelope ---
structure ../01_protein/zika_env_full_centered.pdb
  number 1
  fixed 0.0 0.0 0.0 0.0 0.0 0.0
  fscale 100
end structure

# ---------- OUTER LEAFLET ----------
structure ../02_lipids/POPC.pdb
  number 2160
  radius 1.8
  inside sphere 0.0 0.0 0.0 210.0
  outside sphere 0.0 0.0 0.0 140.0
  atoms {PHOS_ATOM_INDEX}
    inside sphere 0.0 0.0 0.0 192.0
    outside sphere 0.0 0.0 0.0 190.0
  end atoms
  atoms {TAIL_ATOM_INDEX}
    inside sphere 0.0 0.0 0.0 179.0
  end atoms
end structure

structure ../02_lipids/POPE.pdb
  number 1080
  radius 1.8
  inside sphere 0.0 0.0 0.0 210.0
  outside sphere 0.0 0.0 0.0 140.0
  atoms {PHOS_ATOM_INDEX}
    inside sphere 0.0 0.0 0.0 192.0
    outside sphere 0.0 0.0 0.0 190.0
  end atoms
  atoms {TAIL_ATOM_INDEX}
    inside sphere 0.0 0.0 0.0 179.0
  end atoms
end structure

structure ../02_lipids/POPS.pdb
  number 360
  radius 1.8
  inside sphere 0.0 0.0 0.0 210.0
  outside sphere 0.0 0.0 0.0 140.0
  atoms {PHOS_ATOM_INDEX}
    inside sphere 0.0 0.0 0.0 192.0
    outside sphere 0.0 0.0 0.0 190.0
  end atoms
  atoms {TAIL_ATOM_INDEX}
    inside sphere 0.0 0.0 0.0 179.0
  end atoms
end structure

# ---------- INNER LEAFLET ----------
structure ../02_lipids/POPC.pdb
  number 2880
  radius 1.8
  inside sphere 0.0 0.0 0.0 200.0
  outside sphere 0.0 0.0 0.0 120.0
  atoms {PHOS_ATOM_INDEX}
    inside sphere 0.0 0.0 0.0 158.0
    outside sphere 0.0 0.0 0.0 156.0
  end atoms
  atoms {TAIL_ATOM_INDEX}
    outside sphere 0.0 0.0 0.0 169.0
  end atoms
end structure

structure ../02_lipids/POPE.pdb
  number 1440
  radius 1.8
  inside sphere 0.0 0.0 0.0 200.0
  outside sphere 0.0 0.0 0.0 120.0
  atoms {PHOS_ATOM_INDEX}
    inside sphere 0.0 0.0 0.0 158.0
    outside sphere 0.0 0.0 0.0 156.0
  end atoms
  atoms {TAIL_ATOM_INDEX}
    outside sphere 0.0 0.0 0.0 169.0
  end atoms
end structure

structure ../02_lipids/POPS.pdb
  number 480
  radius 1.8
  inside sphere 0.0 0.0 0.0 200.0
  outside sphere 0.0 0.0 0.0 120.0
  atoms {PHOS_ATOM_INDEX}
    inside sphere 0.0 0.0 0.0 158.0
    outside sphere 0.0 0.0 0.0 156.0
  end atoms
  atoms {TAIL_ATOM_INDEX}
    outside sphere 0.0 0.0 0.0 169.0
  end atoms
end structure

cycle2 = ftolerance 3.0
filetype pdb
output zika_pack_cycle2.pdb

packall
movebadrandom

maxit 40
nloop 20
movefrac 0.005

use_short_tol
short_tol_dist 0.5
short_tol_scale 3

restart_from zika_pack_cycle1.restart
restart_to   zika_pack_cycle2.restart

structure ../01_protein/zika_env_full_centered.pdb
  number 1
  fixed 0.0 0.0 0.0 0.0 0.0 0.0
  fscale 500
end structure

# --- all lipid blocks are identical to cycle 1 ---
# Copy/paste the OUTER and INNER leaflet blocks from pack_cycle1.inp here.

Path("zika_pack/03_packmol").mkdir(parents=True, exist_ok=True)
Path("zika_pack/03_packmol/pack_cycle1.inp").write_text(cycle1)
Path("zika_pack/03_packmol/pack_cycle2.inp").write_text(cycle2)
print("Wrote pack_cycle1.inp and pack_cycle2.inp")


## Step 6 — Run Packmol (cycle 1)


In [ ]:
import shutil, subprocess
from pathlib import Path

if shutil.which("packmol"):
    subprocess.check_call("packmol < pack_cycle1.inp | tee pack_cycle1.log", shell=True, cwd=Path("zika_pack/03_packmol"))
else:
    print("Packmol not found; skipping Packmol run.")


## Step 6b — Run Warp-pack (cycle 1)


In [ ]:
from warp_md.pack import parse_inp, run, export

cfg = parse_inp("zika_pack/03_packmol/pack_cycle1.inp")
result = run(cfg)
export(result, "pdb", "zika_pack/03_packmol/zika_pack_cycle1.warp.pdb")


## Step 7 — Run Packmol (cycle 2)


In [ ]:
import shutil, subprocess
from pathlib import Path

if shutil.which("packmol"):
    subprocess.check_call("packmol < pack_cycle2.inp | tee pack_cycle2.log", shell=True, cwd=Path("zika_pack/03_packmol"))
else:
    print("Packmol not found; skipping Packmol run.")


## Step 7b — Run Warp-pack (cycle 2)


In [ ]:
from warp_md.pack import parse_inp, run, export

cfg = parse_inp("zika_pack/03_packmol/pack_cycle2.inp")
result = run(cfg)
export(result, "pdb", "zika_pack/03_packmol/zika_pack_cycle2.warp.pdb")


## Quick comparison checks


In [ ]:
from pathlib import Path

packmol_out = Path("zika_pack/03_packmol/zika_pack_cycle2.pdb")
warp_out = Path("zika_pack/03_packmol/zika_pack_cycle2.warp.pdb")

def count_atoms(path):
    if not path.exists():
        return None
    return sum(1 for line in path.read_text().splitlines() if line.startswith(("ATOM", "HETATM")))

print("Packmol atoms:", count_atoms(packmol_out))
print("Warp-pack atoms:", count_atoms(warp_out))


## Optional QC (MDAnalysis)

Compute min lipid–protein distance.


In [ ]:
try:
    import MDAnalysis as mda
    from MDAnalysis.lib.distances import distance_array
except Exception as exc:
    print("MDAnalysis not available:", exc)
else:
    u = mda.Universe("zika_pack/03_packmol/zika_pack_cycle2.warp.pdb")
    protein = u.select_atoms("protein")
    lipids = u.select_atoms("resname POPC POPE POPS")
    if len(protein) and len(lipids):
        d = distance_array(protein.positions, lipids.positions, backend="OpenMP")
        print("Min protein-lipid distance:", d.min())


## Visualize Structures

Use py3Dmol to view the packed structures interactively.

In [ ]:
try:
    import py3Dmol
    from pathlib import Path
except Exception as exc:
    print("py3Dmol not available:", exc)
else:
    def view_pdb(pdb_file):
        with open(pdb_file, 'r') as f:
            pdb_data = f.read()
        view = py3Dmol.view(width=800, height=600)
        view.addModel(pdb_data, 'pdb')
        view.setStyle({'protein': {'cartoon': {'color': 'blue'}}, 'resname POPC POPE POPS': {'stick': {}}})
        view.zoomTo()
        return view

    # View Packmol output
    packmol_pdb = Path("zika_pack/03_packmol/zika_pack_cycle2.pdb")
    if packmol_pdb.exists():
        print("Packmol output:")
        view_packmol = view_pdb(str(packmol_pdb))
        view_packmol.show()
    else:
        print("Packmol output not found.")

    # View Warp-pack output
    warp_pdb = Path("zika_pack/03_packmol/zika_pack_cycle2.warp.pdb")
    if warp_pdb.exists():
        print("Warp-pack output:")
        view_warp = view_pdb(str(warp_pdb))
        view_warp.show()
    else:
        print("Warp-pack output not found.")

## Download outputs (Colab)

If running in Colab, this cell downloads the pack outputs.


In [ ]:
from pathlib import Path

try:
    from google.colab import files
except Exception as exc:
    print("Not in Colab:", exc)
else:
    outputs = [
        Path("zika_pack/03_packmol/zika_pack_cycle1.warp.pdb"),
        Path("zika_pack/03_packmol/zika_pack_cycle2.warp.pdb"),
        Path("zika_pack/03_packmol/zika_pack_cycle1.pdb"),
        Path("zika_pack/03_packmol/zika_pack_cycle2.pdb"),
    ]
    for path in outputs:
        if path.exists():
            files.download(str(path))
        else:
            print("Missing:", path)
